#cleaned-up TF-IDF approach

#keyword processing

In [1]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langdetect

In [3]:
import pandas as pd
import numpy as np

# keyword extraction and clean up
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

#Text Blob for sentiment analysis
from textblob import TextBlob

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import linear_kernel

import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

In [4]:
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"

In [5]:
hotel_df = pd.read_csv(url)

In [6]:
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive


In [7]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512470 entries, 0 to 512469
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Unnamed: 0                                  512470 non-null  int64  
 1   Hotel_Address                               512470 non-null  object 
 2   Additional_Number_of_Scoring                512470 non-null  int64  
 3   Review_Date                                 512470 non-null  object 
 4   Average_Score                               512470 non-null  float64
 5   Hotel_Name                                  512470 non-null  object 
 6   Reviewer_Nationality                        512470 non-null  object 
 7   Negative_Review                             512470 non-null  object 
 8   Review_Total_Negative_Word_Counts           512470 non-null  int64  
 9   Total_Number_of_Reviews                     512470 non-null  int64  
 

In [8]:
# combine reviews into one field
hotel_df["All_Reviews"] = hotel_df['Positive_Review'].astype(str) +"-"+ hotel_df["Negative_Review"]
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive,No real complaints the hotel was great great ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive,Location was good and staff were ok It is cut...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative,Great location in nice surroundings the bar a...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive,Amazing location and building Romantic settin...


In [9]:
#create random sample of original dataset - 5% sample
hotel_sample_df = hotel_df.sample(frac=.05)

In [10]:
hotel_sample_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
58095,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,55,4,9.2,"[' Leisure trip ', ' Couple ', ' Junior Suite ...",394 day,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...
270866,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,7,4,7.9,"[' Leisure trip ', ' Couple ', ' Deluxe Twin R...",676 day,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...
279084,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,11,3,10.0,"[' Leisure trip ', ' Group ', ' Double or Twin...",710 day,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...
330938,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,22,1,9.2,"[' Leisure trip ', ' Family with older childre...",514 day,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...
252980,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,10,2,8.8,"[' Leisure trip ', ' Couple ', ' Deluxe Double...",104 day,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...


In [11]:
hotel_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25624 entries, 58095 to 406837
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  25624 non-null  int64  
 1   Hotel_Address                               25624 non-null  object 
 2   Additional_Number_of_Scoring                25624 non-null  int64  
 3   Review_Date                                 25624 non-null  object 
 4   Average_Score                               25624 non-null  float64
 5   Hotel_Name                                  25624 non-null  object 
 6   Reviewer_Nationality                        25624 non-null  object 
 7   Negative_Review                             25624 non-null  object 
 8   Review_Total_Negative_Word_Counts           25624 non-null  int64  
 9   Total_Number_of_Reviews                     25624 non-null  int64  
 10  Posit

In [25]:
hotel_sample_df = hotel_sample_df.reset_index(drop=True)
hotel_sample_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,55,4,9.2,"[' Leisure trip ', ' Couple ', ' Junior Suite ...",394 day,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...
1,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,7,4,7.9,"[' Leisure trip ', ' Couple ', ' Deluxe Twin R...",676 day,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...
2,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,11,3,10.0,"[' Leisure trip ', ' Group ', ' Double or Twin...",710 day,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...
3,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,22,1,9.2,"[' Leisure trip ', ' Family with older childre...",514 day,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...
4,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,10,2,8.8,"[' Leisure trip ', ' Couple ', ' Deluxe Double...",104 day,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...


In [26]:
# Change data structure
#hotel_sample_df.drop(['Unnamed: 0', 'Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Positive_Review', 'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags', 'days_since_review'], axis=1) 

In [27]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from combined reviews
hotel_sample_pp_df = hotel_sample_df.copy()
key_words = []
name_key_words = []

r = Rake()

for index, row in hotel_sample_pp_df.iterrows():
    r.extract_keywords_from_text(row['All_Reviews'])
    key_words_dict_scores = r.get_word_degrees()
    keywords = list(key_words_dict_scores.keys())
    
    key_words.append(keywords)

    r.extract_keywords_from_text(row['Hotel_Name'])
    key_words_dict_scores = r.get_word_degrees()
    name_keywords = list(key_words_dict_scores.keys())
    
    name_key_words.append(name_keywords)
    
hotel_sample_pp_df["key_words"] = key_words
hotel_sample_pp_df["name_key_words"] = name_key_words

hotel_sample_pp_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words
0,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,9.2,"[' Leisure trip ', ' Couple ', ' Junior Suite ...",394 day,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...,"[new, modern, hotel, super, clean, picky, get,...","[montcalm, shoreditch, london, tech, city]"
1,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,7.9,"[' Leisure trip ', ' Couple ', ' Deluxe Twin R...",676 day,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...,"[nice, big, room, comfortable, beds, ask, mugs...","[apollofirst, boutique, hotel]"
2,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,10.0,"[' Leisure trip ', ' Group ', ' Double or Twin...",710 day,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...,"[clean, helpful, staff, lovely, breakfast, clo...","[catalonia, atenas]"
3,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,9.2,"[' Leisure trip ', ' Family with older childre...",514 day,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...,"[breakfast, selection, wondeful, staff, helpfu...","[catalonia, passeig, de, gr, cia, 4, sup]"
4,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,8.8,"[' Leisure trip ', ' Couple ', ' Deluxe Double...",104 day,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...,"[decor, friendly, staff, attention, detail, br...","[hoxton, shoreditch]"


In [28]:
hotel_sample_pp_clean_df = hotel_sample_pp_df.copy()

In [29]:
# clean up keywords: drop symbols and numbers, as well as find root words

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


hotel_sample_pp_clean_df['clean_keywords']=hotel_sample_pp_clean_df['key_words'].map(lambda s:preprocess(s)) 
hotel_sample_pp_clean_df.head()


,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords
0,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,"[' Leisure trip ', ' Couple ', ' Junior Suite ...",394 day,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...,"[new, modern, hotel, super, clean, picky, get,...","[montcalm, shoreditch, london, tech, city]",new modern hotel super clean picky get good pr...
1,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,"[' Leisure trip ', ' Couple ', ' Deluxe Twin R...",676 day,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...,"[nice, big, room, comfortable, beds, ask, mugs...","[apollofirst, boutique, hotel]",nice big room comfortable beds ask mugs towels...
2,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,"[' Leisure trip ', ' Group ', ' Double or Twin...",710 day,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...,"[clean, helpful, staff, lovely, breakfast, clo...","[catalonia, atenas]",clean helpful staff lovely breakfast close met...
3,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,"[' Leisure trip ', ' Family with older childre...",514 day,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...,"[breakfast, selection, wondeful, staff, helpfu...","[catalonia, passeig, de, gr, cia, 4, sup]",breakfast selection wondeful staff helpful loc...
4,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,"[' Leisure trip ', ' Couple ', ' Deluxe Double...",104 day,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...,"[decor, friendly, staff, attention, detail, br...","[hoxton, shoreditch]",decor friendly staff attention detail breakfas...


#sentiment analysis

In [30]:
hotel_sub_df = hotel_sample_pp_clean_df.copy()

In [31]:
# run text blob in a loop

polarities = []
subjectivities = []

for index, row in hotel_sub_df.iterrows():
    sentence = row['All_Reviews']
    analysis = TextBlob(sentence).sentiment
    polarity = analysis[0]
    subjectivity = analysis[1]

    # populate lists
    polarities.append(polarity)
    subjectivities.append(subjectivity)


hotel_sub_df['sentiment_polarity'] = polarities
hotel_sub_df['sentiment_subjectivity']= subjectivities

hotel_sub_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...,"[new, modern, hotel, super, clean, picky, get,...","[montcalm, shoreditch, london, tech, city]",new modern hotel super clean picky get good pr...,0.412554,0.599254
1,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...,"[nice, big, room, comfortable, beds, ask, mugs...","[apollofirst, boutique, hotel]",nice big room comfortable beds ask mugs towels...,0.333333,0.633333
2,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...,"[clean, helpful, staff, lovely, breakfast, clo...","[catalonia, atenas]",clean helpful staff lovely breakfast close met...,0.304167,0.537500
3,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...,"[breakfast, selection, wondeful, staff, helpfu...","[catalonia, passeig, de, gr, cia, 4, sup]",breakfast selection wondeful staff helpful loc...,0.027500,0.654375
4,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...,"[decor, friendly, staff, attention, detail, br...","[hoxton, shoreditch]",decor friendly staff attention detail breakfas...,0.375000,0.500000


#TF-IDF

In [32]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [33]:
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

#calculate cosine matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(hotel_sub_df['Hotel_Name'])

C:\Users\neil.ralston.MAC\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


[[1.         0.         0.11741538 ... 0.03682221 0.02439472 0.        ]
 [0.         1.         0.         ... 0.         0.         0.01830963]
 [0.11741538 0.         1.         ... 0.         0.06722919 0.        ]
 ...
 [0.03682221 0.         0.         ... 1.         0.         0.        ]
 [0.02439472 0.         0.06722919 ... 0.         1.         0.03241957]
 [0.         0.01830963 0.         ... 0.         0.03241957 1.        ]]


In [34]:
#get recommendations based on tf-idf and cosine_similarities (hotel name)

def recommend_name(Hotel_Name, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == Hotel_Name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [35]:
recommend_name('Hotel Arena')

,Hotel,Address,Rating,Score
0,Grand Royale London Hyde Park,1 Inverness Terrace Westminster Borough London...,7.7,0.605610
1,Copthorne Tara Hotel London Kensington,Scarsdale Place Kensington Kensington and Chel...,8.1,0.591838
2,Hilton London Paddington,146 Praed Street Westminster Borough London W2...,8.2,0.591838
3,Gran Hotel Barcino,Jaume I 6 Ciutat Vella 08002 Barcelona Spain,7.0,0.581864
4,Hotel Da Vinci,Via Senigallia 6 20161 Milan Italy,8.1,0.573159
5,Park Plaza London Riverbank,18 Albert Embankment Lambeth London SE1 7TJ UK,8.3,0.569652
6,Park Plaza London Riverbank,18 Albert Embankment Lambeth London SE1 7TJ UK,8.3,0.532631
7,Pullman Paris Centre Bercy,1 Rue De Libourne 12th arr 75012 Paris France,8.7,0.530834
8,The Belgrave Hotel,80 86 Belgrave Road Westminster Borough London...,8.6,0.503336
9,Sofitel London St James,6 Waterloo Place Westminster Borough London SW...,9.2,0.482722


In [36]:
#get recommendations based on tf-idf and cosine_similarities (keyword)

def recommend_keyword(clean_keywords, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == clean_keywords].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec2_df = pd.DataFrame(data)
    
    return rec2_df

In [37]:
recommend_keyword('location')

IndexError: index 0 is out of bounds for axis 0 with size 0

#first review per hotel

In [47]:
hotel_unique_df = hotel_sub_df.drop_duplicates('Hotel_Name')

In [59]:
hotel_unique_df

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,58095,151 157 City Road Shoreditch Islington London ...,1471,2016-07-05,9.1,M by Montcalm Shoreditch London Tech City,United States of America,No Negative,0,4802,...,51.527847,-0.088947,UK,Positive,Very new and modern hotel Super clean if you ...,"[new, modern, hotel, super, clean, picky, get,...","[montcalm, shoreditch, london, tech, city]",new modern hotel super clean picky get good pr...,0.412554,0.599254
1,271248,Apollolaan 123 125 Oud Zuid 1077 AP Amsterdam ...,181,2015-09-27,8.5,Apollofirst Boutique Hotel,United Kingdom,Had to ask for mugs in room no t towels in ki...,15,1284,...,52.350396,4.875640,Netherlands,Positive,Nice big room with comfortable beds- Had to a...,"[nice, big, room, comfortable, beds, ask, mugs...","[apollofirst, boutique, hotel]",nice big room comfortable beds ask mugs towels...,0.333333,0.633333
2,279466,Avenida Meridiana 151 Sant Mart 08026 Barcelon...,541,2015-08-24,7.9,Catalonia Atenas,United Kingdom,No Negative,0,6792,...,41.411006,2.186601,Spain,Positive,clean very helpful staff lovely breakfast Clo...,"[clean, helpful, staff, lovely, breakfast, clo...","[catalonia, atenas]",clean helpful staff lovely breakfast close met...,0.304167,0.537500
3,331559,Gran Via de les Corts Catalanes 644 Eixample 0...,257,2016-03-07,9.2,Catalonia Passeig de Gr cia 4 Sup,United Kingdom,The evening meal in the restaurant was tastel...,38,2116,...,41.390140,2.169670,Spain,Positive,The breakfast selection as wondeful the staff...,"[breakfast, selection, wondeful, staff, helpfu...","[catalonia, passeig, de, gr, cia, 4, sup]",breakfast selection wondeful staff helpful loc...,0.027500,0.654375
4,253362,81 Great Eastern Street Islington London EC2A ...,524,2017-04-21,9.0,The Hoxton Shoreditch,United Kingdom,Nothing,2,1732,...,51.525514,-0.082927,UK,Positive,Decor friendly staff attention to detail brea...,"[decor, friendly, staff, attention, detail, br...","[hoxton, shoreditch]",decor friendly staff attention detail breakfas...,0.375000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24624,323268,Felberstra e 4 15 Rudolfsheim F nfhaus 1150 Vi...,54,2016-12-25,7.9,Hotel Mercure Wien Westbahnhof,Albania,The organization,3,842,...,48.198354,16.337525,Austria,Positive,The bed was comfy and breakfast was spr tasty...,"[bed, comfy, breakfast, spr, tasty, left, desi...","[hotel, mercure, wien, westbahnhof]",bed comfy breakfast spr tasty left desire orga...,0.250000,0.250000
24659,162805,36 rue de Saint Quentin 10th arr 75010 Paris F...,11,2017-07-14,9.2,Hotel Whistler,Canada,No Negative,0,115,...,48.879314,2.356018,France,Positive,Very nice staff that make you feel welcome an...,"[nice, staff, make, feel, welcome, give, life,...","[hotel, whistler]",nice staff make feel welcome give life hotel t...,0.324872,0.596538
25054,122610,251 rue Saint Honor 1st arr 75001 Paris France,10,2015-11-01,9.1,Mandarin Oriental Paris,France,Not new hotel decorechion not new so i belive...,18,81,...,48.867189,2.326927,France,Positive,Very good hotel clean very nice wie from wind...,"[good, hotel, clean, nice, wie, window, person...","[mandarin, oriental, paris]",good hotel clean nice wie window personal star...,0.275556,0.493220
25178,201793,5 Rue de Montyon 9th arr 75009 Paris France,19,2016-11-15,7.5,Mercure Paris Op ra Faubourg Montmartre,Azerbaijan,Small room,3,223,...,48.872964,2.344490,France,Positive,No Positive- Small room,"[positive, small, room]","[mercure, paris, op, ra, faubourg, montmartre]",positive small room,-0.181818,0.472727


In [62]:
hotel_unique_df2 = hotel_unique_df.drop(['Unnamed: 0', 'Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Review_Total_Positive_Word_Counts', 'Reviewer_Score', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags', 'lat', 'lng', 'days_since_review', 'Sentiment', 'All_Reviews'], axis=1) 

In [63]:
hotel_unique_df2.sort_values(by=['Average_Score'], ascending=False)

,Hotel_Address,Average_Score,Hotel_Name,Negative_Review,Positive_Review,Hotel_Country,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
1704,15 Place Vend me 1st arr 75001 Paris France,9.8,Ritz Paris,No Negative,Great place,France,"[great, place, negative]","[ritz, paris]",great place negative,0.475000,0.575000
2237,Passeig de Colom 9 Ciutat Vella 08002 Barcelon...,9.6,Hotel The Serras,The bathroom wall is opaque glass which didn ...,The staff the location the style and design T...,Spain,"[staff, location, style, design, petit, fours,...","[hotel, serras]",staff location style design petit fours water ...,0.466667,0.675000
5798,Pau Claris 179 Eixample 08037 Barcelona Spain,9.6,H10 Casa Mimosa 4 Sup,No Negative,Staff very helpful Room overlooked rear garde...,Spain,"[staff, helpful, room, overlooked, rear, garde...","[h10, casa, mimosa, 4, sup]",staff helpful room overlooked rear garden casa...,0.310000,0.496667
6581,41 Buckingham Palace Road Westminster Borough ...,9.6,41,Details of room Hard to get to plugs shower i...,Excellent location Attentive courteous staff,UK,"[excellent, location, attentive, courteous, st...",[41],excellent location attentive courteous staff d...,0.011083,0.496750
3673,Elisabets 11 Ciutat Vella 08001 Barcelona Spain,9.6,Hotel Casa Camper,No Negative,We can t praise this hotel enough The staff w...,Spain,"[praise, hotel, enough, staff, extremely, help...","[hotel, casa, camper]",praise hotel enough staff extremely helpful fr...,0.195000,0.581667
...,...,...,...,...,...,...,...,...,...,...,...
459,28 Ebury Street Westminster Borough London SW1...,6.6,The Tophams Hotel,Bathroom taps for some reason all hot even th...,Welcoming friendly staff good room and facili...,UK,"[welcoming, friendly, staff, good, room, facil...","[tophams, hotel]",welcoming friendly staff good room facilities ...,0.165000,0.710000
6221,346 348 Edgware Road Westminster Borough Londo...,6.6,Best Western Maitrise Hotel Edgware Road,The rooms were ridiculously tiny the dressing...,Nothing,UK,"[nothing, rooms, ridiculously, tiny, dressing,...","[best, western, maitrise, hotel, edgware, road]",nothing rooms ridiculously tiny dressing table...,-0.092157,0.583660
27,Ferdinand Bolstraat 194 Oud Zuid 1072 LW Amste...,6.4,Savoy Hotel Amsterdam,very disappointing in every way,No Positive,Netherlands,"[positive, disappointing, every, way]","[savoy, hotel, amsterdam]",positive disappointing every way,-0.446818,0.727727
248,75 Gower Street Camden London WC1E 6HJ UK,6.4,Hotel Cavendish,Staff had poor level of English Thin walls TV...,Convenient location Free breakfast,UK,"[convenient, location, free, breakfast, staff,...","[hotel, cavendish]",convenient location free breakfast staff poor ...,-0.066667,0.408333
